# TMDB Tensor Reranking

Reranks TMDB Solr search results by BERT output tensor similarity

In [2]:
import spacy
import torch
import numpy
import json
import datetime
import sys
import pickle
import pysolr
from IPython.display import Markdown, display
from numpy.testing import assert_almost_equal
nlp = spacy.load("en_trf_bertbaseuncased_lg")

## Solr Query Utils

Used to query solr and print fancy markdown results to the notebook

In [3]:
#Solr Client
solr = pysolr.Solr('http://localhost:8983/solr/tmdb')

#Formats a search result as markdown and prints
def printmd(movieid,title,date,overview,score):
    title = '### ' + title + ' (' + str(movieid) + ')\n'
    score = '__' + str(score) + '__\n'
    date = '_' + str(date) + '_\n'
    overview = '> ' + overview
    display(Markdown(title))
    display(Markdown(score))
    display(Markdown(date))
    display(Markdown(overview))

#Print the Title, Release Date, and Overview from TMDB
def printresults(res):
    for r in res:
        printmd(r["id"],r["title"][0],r["release_date"],r["overview"][0],r["likeness"])

#Search the title and overview fields for a given query text, with a small title boost
def solrquery(q):
    return "overview:("+q+")"
        
#Enrich and Search a text query
def search(query):
    q = solrquery(query)
    print("Searching for `" + query + "` ...")
    print("  q = " + q)
    res = solr.search(q=q,fl="id,title,overview,release_date,score",rows=100)
    return res

## Rerank by tensor similarity

This method will take the top 100 results from the plain solr query, then perform a similarity operation on the query to each of those results.  The results are then reranked in order of this similarity.

In [4]:
def similarity(t1,t2):
    s = 1. / (1. + (W * (vec1-vec2)**2).sum())
    return s

In [5]:
def similarity_manual(tensor1,tensor2):
    total=0
    dims=0
    for a in tensor1:
        for b in tensor2:
            total+=a.dot(b)
            dims+=1
    return total/dims

In [6]:
def search_and_rerank(querytext):
    res = search(querytext)
    query = nlp(querytext).tensor
    ids = [str(r["id"]) for r in res]
    for r in res:
        movieid=str(r["id"])
        tensor=None
        likeness=0
        try:
            with open("vectors/"+movieid+".pickle", "rb") as input_file:
                 doc = pickle.load(input_file)
        except:
            r["overview"] = [""]
            #print(movieid,"doesn't have a tensor!")
            
        if doc is not None:
            likeness = similarity_manual(doc,query)
            #print(distance)
        r["likeness"] = likeness
    reranked = sorted(res, reverse=True, key=lambda k: k['likeness'])
    return reranked[:100]

In [8]:
res = search_and_rerank("angry robot")
printresults(res)

Searching for `angry robot` ...
  q = overview:(angry robot)


### Dragon Ball Z: Return of Cooler (39103)


__96.59150388966437__


_1992-03-07T00:00:00Z_


> Cooler has resurrected himself as a robot and is enslaving the people of New Namek. Goku and the gang must help.

...

> Lawrence is a rich kid with a bad accent and a large debt. After his father refuses to help him out, Lawrence escapes his angry debtors by jumping on a Peace Corp flight to Southeast Asia, where he is assigned to build a bridge for the local villagers with American-As-Apple-Pie WSU Grad Tom Tuttle and the beautiful and down-to earth Beth Wexler.

### Our Little Differences (128151)


__60.87947003316071__


_2012-07-02T00:00:00Z_


> In this acutely observed and sharply written debut feature from director Sylvie Michel, the contrasting parenting philosophies of a well-to-do divorcee and his immigrant cleaning lady becomes an angry debate over deeper issues of social class and cultural values when their teenage children disappear after a night on the town. (TIFF)